# CIBERSORTx Results Analysis

In [1]:
import sys 
%load_ext rpy2.ipython
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
import copy
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import anndata as ad
import scipy
import seaborn as sns
sc.set_figure_params(scanpy=True, dpi_save=200)
import jenkspy

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [2]:
def load_fractions(dem_path):
    def scale_series(series): 
        return((series - np.mean(series))/np.std(series))
    def scale_series_mad(series): 
        return((series - np.median(series))/scipy.stats.median_absolute_deviation(series))

    # load CIBERSORT output
    dat = sc.read_csv(dem_path, first_column_names=True)
    dat = dat[:,~dat.var_names.isin(['P-value', 'Correlation', 'RMSE', 'Absolute score (sig.score)'])]
    dat.obs.index.rename('Sample', inplace=True)

    # add the absolute scores
    cell_type_scores = pd.DataFrame(dat.X, index=dat.obs_names, columns=dat.var_names+"-Absolute")
    # Yields a tuple of column name and series for each column in the dataframe
    for (columnName, columnData) in cell_type_scores.iteritems():
        cell_type_scores[columnName.replace('Absolute', 'Abs.Category')] = np.where(columnData.values > np.median(columnData.values), "High", "Low")

    dat.obs = dat.obs.merge(cell_type_scores, how = 'left', left_index=True, right_index=True)

    # Relative Scores (tumor only)
    relative_scores = pd.DataFrame(dat.X, index=dat.obs_names, columns=dat.var_names+"-Relative")
    relative_scores = (relative_scores.transpose() / relative_scores.sum(axis=1).transpose()).transpose()
    for (columnName, columnData) in relative_scores.iteritems():
        relative_scores[columnName.replace('Relative', 'Rel.Category')] = np.where(columnData.values > np.median(columnData.values), "High", "Low")
    dat.obs = dat.obs.merge(relative_scores, how = 'left', left_index=True, right_index=True)
    
    return dat

In [5]:
PC2_loadings = hcat.varm['PCs'][:,1]
PC2_loadings = np.array([-0.20023263, -0.08772767, -0.27906924, -0.5126374, 0.7101968, 0.04527085, 0.32419923])

## Loading Prediction Output

In [7]:
def get_ranklist(dat, classification, cluster_A, cluster_B, test='wilcoxon'):
    
    adata = dat[dat.obs[classification].isin([cluster_A, cluster_B])]

    sc.tl.rank_genes_groups(adata, classification, method = test, n_genes=20000)
    sam_cat_DE_df = pd.concat(
            [pd.DataFrame(adata.uns['rank_genes_groups']['names']),
             pd.DataFrame(adata.uns['rank_genes_groups']['scores']),
             pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges']),
             pd.DataFrame(adata.uns['rank_genes_groups']['pvals'])],
        keys = ['names','scores','logfoldchanges', 'pvals'],axis=1)

    sam_cat_DE_df.columns = sam_cat_DE_df.columns.swaplevel(0, 1)
    sam_cat_DE_df.sort_index(axis=1,level=0,inplace=True)
    sam_cat_DE_df = sam_cat_DE_df[cluster_A][['names', 'scores', 'logfoldchanges', 'pvals']]

    return(sam_cat_DE_df)

In [8]:
def get_scores(df, dat, classification, cluster_A, cluster_B, cohort, drug, target):
    # Absolute Scores
        
    ## Wilcox
    temp = get_ranklist(dat, classification, cluster_A, cluster_B, test='wilcoxon')
    temp['Cohort'] = cohort
    temp['Drug'] = drug
    temp['Target'] = target
    temp['Test'] = 'wilcox'
    df = df.append(temp)
    
    return(df)

# Run on Each Dataset

In [9]:
df = pd.DataFrame()

## 5 Inhibitor

In [10]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_5inhibitor_AF9.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs.index = dat.obs.index.str.replace('.','_')
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')
dat.obs['Timepoint'] = [x[2] for x in dat.obs.index.str.split('_')]
dat.obs['Line'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Cohort'] = "5_inhibitors"

## 5 Inhibitor nonAF9

In [11]:
dat2 = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_5inhibitor_nonAF9.csv")
dat2.X = (dat2.X.transpose() / dat2.X.sum(axis=1)).transpose()
dat2.obs.index = dat2.obs.index.str.replace('.','_')
dat2.obs['Treatment'] = [x[1] for x in dat2.obs.index.str.split('_')]
dat2.obs['Treatment'] = dat2.obs['Treatment'].str.replace('DMSO','Control')
dat2.obs['Timepoint'] = [x[2] for x in dat2.obs.index.str.split('_')]
dat2.obs['Line'] = [x[0] for x in dat2.obs.index.str.split('_')]
dat2.obs['Cohort'] = "5_inhibitors"

In [12]:
dat = dat.concatenate(dat2)
del(dat2)

In [13]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [14]:
# MLL-AF9
for line in ['MOLM13', 'MOLM14', 'OCIAML2', 'NOMO1']:
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'Brequinar', 'Control', line, 'Brequinar', 'DHODH')
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'BAY155', 'Control', line, 'BAY155', 'Menin-MLL')
    df = get_scores(df, dat[(dat.obs.Timepoint == '96h') & (dat.obs.Line == line)], 'Treatment', 'EPZ5676', 'Control', line, 'EPZ5676', 'DOT1L')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'OTX015', 'Control', line, 'OTX015', 'BET')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'BAY1251152', 'Control', line, 'BAY1251152', 'CDK9')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [15]:
# MLL, non-AF9
for line in ['ML2', 'MV411']:
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'Brequinar', 'Control', line, 'Brequinar', 'DHODH')
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'BAY155', 'Control', line, 'BAY155', 'Menin-MLL')
    df = get_scores(df, dat[(dat.obs.Timepoint == '96h') & (dat.obs.Line == line)], 'Treatment', 'EPZ5676', 'Control', line, 'EPZ5676', 'DOT1L')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'OTX015', 'Control', line, 'OTX015', 'BET')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'BAY1251152', 'Control', line, 'BAY1251152', 'CDK9')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [16]:
# Non-MLL
for line in ['OCIAML5', 'HL60', 'OCIAML3']:
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'Brequinar', 'Control', line, 'Brequinar', 'DHODH')
    df = get_scores(df, dat[(dat.obs.Timepoint == '24h') & (dat.obs.Line == line)], 'Treatment', 'BAY155', 'Control', line, 'BAY155', 'Menin-MLL')
    df = get_scores(df, dat[(dat.obs.Timepoint == '96h') & (dat.obs.Line == line)], 'Treatment', 'EPZ5676', 'Control', line, 'EPZ5676', 'DOT1L')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'OTX015', 'Control', line, 'OTX015', 'BET')
    df = get_scores(df, dat[(dat.obs.Timepoint.isin(['8h', '24h'])) & (dat.obs.Line == line)], 'Treatment', 'BAY1251152', 'Control', line, 'BAY1251152', 'CDK9')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## BET ARV-825 OTX015

In [17]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_BET_PROTAC_OCIAML5.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = np.where(dat.obs.index.str.contains("ARV"), "ARV-825", 
                               np.where(dat.obs.index.str.contains("OTX015"), "OTX015", "Control"))

In [18]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [19]:
df = get_scores(df, dat, 'Treatment', 'ARV-825', 'Control', "OCI-AML5", 'ARV-825', 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [20]:
df = get_scores(df, dat, 'Treatment', 'OTX015', 'Control', "OCI-AML5", 'OTX015', 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## CDK6 Selective

In [21]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_CDK6_Selective.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')

In [22]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [23]:
df = get_scores(df, dat, 'Treatment', 'Palbo', 'Control', "MV4-11", 'Palbociclib', 'CDK4/6')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [24]:
df = get_scores(df, dat, 'Treatment', 'YKL', 'Control', "MV4-11", 'YKL (CDK6/IKZF)', 'CDK6/IKZF')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [25]:
df = get_scores(df, dat, 'Treatment', 'BSJ', 'Control', "MV4-11", 'BSJ (CDK6 only)', 'CDK6 only')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## DHODH 2 Lines

In [26]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_DHODH_2Lines.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs.index = dat.obs.index.str.replace('.', '_')
dat.obs['Line'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')

In [27]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [28]:
df = get_scores(df, dat[dat.obs.Line == 'KG1'], 'Treatment', 'ASLAN003', 'Control', "KG1", 'ASLAN003', 'DHODH')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [29]:
df = get_scores(df, dat[dat.obs.Line == 'MOLM14'], 'Treatment', 'ASLAN003', 'Control', "MOLM-14", 'ASLAN003', 'DHODH')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## TET1 Inhibitor

In [30]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_TET1_inhibitor_THP1.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('-')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('Ctrl','Control')

In [31]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [32]:
df = get_scores(df, dat, 'Treatment', 'NSC', 'Control', "THP-1", 'NSC-370', 'TET1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## STAT5 Inhibitor

In [33]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_STAT5_Inhibitor.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Line'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = [x[3] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')

In [34]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [35]:
df = get_scores(df, dat[dat.obs.Line=='MV4'], 'Treatment', 'AC', 'Control', "MV4-11", 'AC-4-130', 'STAT5')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [36]:
df = get_scores(df, dat[dat.obs.Line=='MOLM'], 'Treatment', 'AC', 'Control', "MOLM-13", 'AC-4-130', 'STAT5')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## MitoBlock6

In [37]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_copperLSC.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[2] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('ctrl','Control')

In [38]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [39]:
df = get_scores(df, dat, 'Treatment', 'MB6', 'Control', "OCI-AML2", 'MitoBlock6', 'Mito Copper')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## KDM1A

In [40]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_KDM1A.csv")
anno = pd.read_csv("GSE71740_RNA-seq_sample_annotations.txt", delimiter='\t').set_index('Sample')
dat.obs = dat.obs.join(anno)
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.SampleTitle.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')
del(anno)

In [41]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [42]:
df = get_scores(df, dat[(dat.obs.Timepoint==24) & (dat.obs['Cell Line'] == 'SK-NO-1')], 'Treatment', 'GSK690', 'Control', "SKNO-1", 'GSK690 (24hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [43]:
df = get_scores(df, dat[(dat.obs.Timepoint==24) & (dat.obs['Cell Line'] == 'SK-NO-1')], 'Treatment', 'RN-1', 'Control', "SKNO-1", 'RN-1 (24hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [44]:
df = get_scores(df, dat[(dat.obs.Timepoint==24) & (dat.obs['Cell Line'] == 'Kasumi-1')], 'Treatment', 'GSK690', 'Control', "Kasumi-1", 'GSK690 (24hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [45]:
df = get_scores(df, dat[(dat.obs.Timepoint==24) & (dat.obs['Cell Line'] == 'Kasumi-1')], 'Treatment', 'RN-1', 'Control', "Kasumi-1", 'RN-1 (24hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [46]:
df = get_scores(df, dat[(dat.obs.Timepoint==72) & (dat.obs['Cell Line'] == 'SK-NO-1')], 'Treatment', 'GSK690', 'Control', "SKNO-1", 'GSK690 (72hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [47]:
df = get_scores(df, dat[(dat.obs.Timepoint==72) & (dat.obs['Cell Line'] == 'SK-NO-1')], 'Treatment', 'RN-1', 'Control', "SKNO-1", 'RN-1 (72hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [48]:
df = get_scores(df, dat[(dat.obs.Timepoint==72) & (dat.obs['Cell Line'] == 'Kasumi-1')], 'Treatment', 'GSK690', 'Control', "Kasumi-1", 'GSK690 (72hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

In [49]:
df = get_scores(df, dat[(dat.obs.Timepoint==72) & (dat.obs['Cell Line'] == 'Kasumi-1')], 'Treatment', 'RN-1', 'Control', "Kasumi-1", 'RN-1 (72hr)', 'KDM1A')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribu

## DOT1L and EPZ004777

In [50]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_MeninMLL.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat = dat[dat.obs.index!='Sum']
dat.obs['Line'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control').str.replace('DFCI', 'EPZ004777')

Trying to set attribute `.obs` of view, copying.


In [51]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [52]:
df = get_scores(df, dat[dat.obs.Line == 'OCI2'], 'Treatment', 'EPZ004777', 'Control', "OCI-AML2", 'EPZ004777', 'DOT1L')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [53]:
df = get_scores(df, dat[dat.obs.Line == 'OCI3'], 'Treatment', 'EPZ004777', 'Control', "OCI-AML3", 'EPZ004777', 'DOT1L')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## FTO Inhibitor NB4

In [54]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_FTO_KD_Inhibitor_NB4.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs.index = dat.obs.index.str.replace('.','_')
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control')

In [55]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [56]:
df = get_scores(df, dat, 'Treatment', '231', 'Control', "NB-4", 'FB23-1', 'FTO')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [57]:
df = get_scores(df, dat, 'Treatment', '232', 'Control', "NB-4", 'FB23-2', 'FTO')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## FTO Inhibitor NOMO

In [58]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Results_FTO_KD_Inhibitor_NOMO.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs.index = dat.obs.index.str.replace('.','_')
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]

In [59]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [60]:
df = get_scores(df, dat, 'Treatment', 'CS1', 'Control', "NOMO-1", 'CS1', 'FTO')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [61]:
df = get_scores(df, dat, 'Treatment', 'CS2', 'Control', "NOMO-1", 'CS2', 'FTO')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## MTHFR BET OTX015

In [62]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_MTHFR_Results.csv")
dat_IMSM2 = dat[0:12]
dat_IMSM2.X = (dat_IMSM2.X.transpose() / dat_IMSM2.X.sum(axis=1)).transpose()
dat_IMSM2.obs['Treatment'] = [x[1] for x in dat_IMSM2.obs.index.str.split('_')]
dat_IMSM2.obs['Treatment'] = dat_IMSM2.obs['Treatment'].str.replace('DMSO','Control')
dat_IMSM2.obs['Treatment'] = dat_IMSM2.obs['Treatment'].str.replace('OTX','OTX015')

Trying to set attribute `.obs` of view, copying.


In [63]:
newdat = dat_IMSM2.to_df()
newdat['PC2'] = (dat_IMSM2.to_df() * PC2_loadings).sum(axis=1)
dat_IMSM2 = ad.AnnData(newdat, dat_IMSM2.obs)
del(newdat)

In [64]:
df = get_scores(df, dat_IMSM2, classification = 'Treatment', cluster_A = 'OTX015', cluster_B = 'Control', cohort = 'IMS-M2', drug = 'OTX015', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [65]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_MTHFR_Results.csv")
dat_U937 = dat[12:]
dat_U937.X = (dat_U937.X.transpose() / dat_U937.X.sum(axis=1)).transpose()
dat_U937.obs['Treatment'] = [x[1] for x in dat_U937.obs.index.str.split('_')]
dat_U937.obs['Treatment'] = dat_U937.obs['Treatment'].str.replace('DMSO','Control')
dat_U937.obs['Treatment'] = dat_U937.obs['Treatment'].str.replace('OTX','OTX015')

Trying to set attribute `.obs` of view, copying.


In [66]:
newdat = dat_U937.to_df()
newdat['PC2'] = (dat_U937.to_df() * PC2_loadings).sum(axis=1)
dat_U937 = ad.AnnData(newdat, dat_U937.obs)
del(newdat)

In [67]:
df = get_scores(df, dat_U937, classification = 'Treatment', cluster_A = 'OTX015', cluster_B = 'Control', cohort = 'U-937', drug = 'OTX015', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## Phospho MEF2C MRT199665

In [68]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_MEF2C_MARK_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x for x in dat.obs.index]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('rep','').str.replace('[0-9]', '')

In [69]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [70]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'MRT', cluster_B = 'Control', cohort = 'OCI-AML3', drug = 'MRT199665', target = 'Phospho MEF2C')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## I-BET152

In [71]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_I_BET152_Results.csv")
dat_K562 = dat[:10]
dat_K562.X = (dat_K562.X.transpose() / dat_K562.X.sum(axis=1)).transpose()
dat_K562.obs['Treatment'] = [x[1] for x in dat_K562.obs.index.str.split('_')]
dat_K562.obs['Treatment'] = dat_K562.obs['Treatment'].str.replace('DMSO', 'Control').str.replace('I5000', 'I-BET152')
dat_K562.obs['Treatment'] = dat_K562.obs['Treatment'].str.replace('I500', 'I-BET152')
dat_K562.obs['Treatment'] = dat_K562.obs['Treatment'].str.replace('I50', 'I-BET152')

Trying to set attribute `.obs` of view, copying.


In [72]:
newdat = dat_K562.to_df()
newdat['PC2'] = (dat_K562.to_df() * PC2_loadings).sum(axis=1)
dat_K562 = ad.AnnData(newdat, dat_K562.obs)
del(newdat)

In [73]:
df = get_scores(df, dat_K562, classification = 'Treatment', cluster_A = 'I-BET152', cluster_B = 'Control', cohort = 'K-562', drug = 'I-BET152', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [74]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_I_BET152_Results.csv")
dat_MV411 = dat[10:]
dat_MV411.X = (dat_MV411.X.transpose() / dat_MV411.X.sum(axis=1)).transpose()
dat_MV411.obs['Treatment'] = [x[1] for x in dat_MV411.obs.index.str.split('_')]
dat_MV411.obs['Treatment'] = dat_MV411.obs['Treatment'].str.replace('DMSO', 'Control').str.replace('I5000', 'I-BET152')
dat_MV411.obs['Treatment'] = dat_MV411.obs['Treatment'].str.replace('I500', 'I-BET152')
dat_MV411.obs['Treatment'] = dat_MV411.obs['Treatment'].str.replace('I50', 'I-BET152')

Trying to set attribute `.obs` of view, copying.


In [75]:
newdat = dat_MV411.to_df()
newdat['PC2'] = (dat_MV411.to_df() * PC2_loadings).sum(axis=1)
dat_MV411 = ad.AnnData(newdat, dat_MV411.obs)
del(newdat)

In [76]:
df = get_scores(df, dat_MV411, classification = 'Treatment', cluster_A = 'I-BET152', cluster_B = 'Control', cohort = 'MV4-11', drug = 'I-BET152', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## MDMX/MDM2 ALRN-6924

In [77]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_MDMX_MDM2_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('Vehicle','Control')

In [78]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [79]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ALRN-6924', cluster_B = 'Control', cohort = 'MOLM-13', drug = 'Dexamathasone', target = 'MDMX/MDM2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## FLT3 + Dexamethasone + Quizartinib

In [80]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_FLT3_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('-')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO','Control').str.replace('Quiz', 'Quizartinib').str.replace('Dex', 'Dexamathasone')

In [81]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [82]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Dexamathasone', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Dexamathasone', target = 'FLT3')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [83]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Quizartinib', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Quizartinib', target = 'FLT3')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [84]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Quizartinib + Dexamathasone', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Quizartinib + Dexamathasone', target = 'FLT3')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## DOT1L EZH2

In [85]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_DOT1LEZH2_MV411_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('NC', 'Control').str.replace('low', '').str.replace('high', '').str.replace('EZH2', 'GSK126').str.replace('DOT1L', 'EPZ5676').str.replace('Combo', 'GSK126 + EPZ5676')

In [86]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [87]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK126', cluster_B = 'Control', cohort = 'MV4-11', drug = 'GSK126', target = 'EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [88]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'EPZ5676', cluster_B = 'Control', cohort = 'MV4-11', drug = 'EPZ5676', target = 'DOT1L')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [89]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK126 + EPZ5676', cluster_B = 'Control', cohort = 'MV4-11', drug = 'EPZ5676 + GSK126', target = 'DOT1L + EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [90]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_DOT1LEZH2_MOLM_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x for x in dat.obs.index]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('Molm', '').str.replace('[0-9]', '').str.replace('DMSO', 'Control').str.replace('DotL', 'EPZ5676').str.replace('Ezh', 'GSK126').str.replace('Combo', 'GSK126 + EPZ5676')

In [91]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [92]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK126', cluster_B = 'Control', cohort = 'MOLM-14', drug = 'GSK126', target = 'EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [93]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'EPZ5676', cluster_B = 'Control', cohort = 'MOLM-14', drug = 'EPZ5676', target = 'DOT1L')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [94]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK126 + EPZ5676', cluster_B = 'Control', cohort = 'MOLM-14', drug = 'EPZ5676 + GSK126', target = 'DOT1L + EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

##  GSK3a/b BRD

In [95]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_GSK3ab_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]

In [96]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [97]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'BRD0705', cluster_B = 'Control', cohort = 'U-937', drug = 'BRD0705', target = 'GSK3a')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [98]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'BRD0320', cluster_B = 'Control', cohort = 'U-937', drug = 'BRD0320', target = 'GSK3a/b')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## POL I BET CX/I-BET/Doxo

In [99]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_POL_I_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x for x in dat.obs.index]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('\.[^.]*$', '').str.replace('[0-9]', '').str.replace('.', '_')
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('v', 'Control').str.replace('ib', 'I-BET').str.replace('dox', 'Doxo').str.replace('cx', 'CX')

In [100]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [101]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'CX', cluster_B = 'Control', cohort = 'MV4-11', drug = 'CX-5461', target = 'POL I')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [102]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Doxo', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Doxorubicin', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [103]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'I-BET', cluster_B = 'Control', cohort = 'MV4-11', drug = 'I-BET151', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [104]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Doxo_I-BET', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Doxorubicin + I-BET151', target = 'BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [105]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'CX_I-BET', cluster_B = 'Control', cohort = 'MV4-11', drug = 'CX-5461 + I-BET151', target = 'POL I + BET')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## Menin-MLL VTP

In [106]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Menin_MLL_Results.csv")
dat_IMSM2 = dat[:18]
dat_IMSM2.X = (dat_IMSM2.X.transpose() / dat_IMSM2.X.sum(axis=1)).transpose()
dat_IMSM2.obs['Treatment'] = [x[2] for x in dat_IMSM2.obs.index.str.split('_')]
dat_IMSM2.obs['Treatment'] = dat_IMSM2.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [107]:
newdat = dat_IMSM2.to_df()
newdat['PC2'] = (dat_IMSM2.to_df() * PC2_loadings).sum(axis=1)
dat_IMSM2 = ad.AnnData(newdat, dat_IMSM2.obs)
del(newdat)

In [108]:
df = get_scores(df, dat_IMSM2, classification = 'Treatment', cluster_A = 'VTP', cluster_B = 'Control', cohort = 'IMS-M2', drug = 'VTP-50469', target = 'Menin-MLL')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [109]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Menin_MLL_Results.csv")
dat_OCIAML3 = dat[18:]
dat_OCIAML3.X = (dat_OCIAML3.X.transpose() / dat_OCIAML3.X.sum(axis=1)).transpose()
dat_OCIAML3.obs['Treatment'] = [x[2] for x in dat_OCIAML3.obs.index.str.split('_')]
dat_OCIAML3.obs['Treatment'] = dat_OCIAML3.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [110]:
newdat = dat_OCIAML3.to_df()
newdat['PC2'] = (dat_OCIAML3.to_df() * PC2_loadings).sum(axis=1)
dat_OCIAML3 = ad.AnnData(newdat, dat_OCIAML3.obs)
del(newdat)

In [111]:
df = get_scores(df, dat_OCIAML3, classification = 'Treatment', cluster_A = 'VTP', cluster_B = 'Control', cohort = 'OCI-AML3', drug = 'VTP-50469', target = 'Menin-MLL')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## LSD1 MTORC1 OG86/RAD001

In [112]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_LSD1_MTORC1_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [113]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [114]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'OG86', cluster_B = 'Control', cohort = 'THP-1', drug = 'OG86', target = 'LSD1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [115]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'RAD001', cluster_B = 'Control', cohort = 'THP-1', drug = 'RAD001', target = 'MTORC1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [116]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'RAD001-OG86', cluster_B = 'Control', cohort = 'THP-1', drug = 'RAD001 + OG86', target = 'LSD1 + MTORC1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## GCN5 LSD1 MB-3/GSK-LSD1/ATRA

In [117]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_GCN5_LSD1_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x for x in dat.obs.index]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('untreated', 'Control')
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('^[^_]*_', '').str.replace('_', '-')

In [118]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [119]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA', cluster_B = 'Control', cohort = 'HL-60', drug = 'ATRA', target = 'RARA')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [120]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'MB3', cluster_B = 'Control', cohort = 'HL-60', drug = 'MB3', target = 'GCN5')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [121]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK-LSD1', cluster_B = 'Control', cohort = 'HL-60', drug = 'GSK-LSD1', target = 'LSD1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [122]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA-MB3', cluster_B = 'Control', cohort = 'HL-60', drug = 'ATRA + MB3', target = 'RARA + GCN-5')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [123]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA-GSK-LSD1', cluster_B = 'Control', cohort = 'HL-60', drug = 'ATRA + GSK-LSD1', target = 'RARA + LSD1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [124]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA-MB3-GSK-LSD1', cluster_B = 'Control', cohort = 'HL-60', drug = 'ATRA + MB3 + GSK-LSD1', target = 'RARA + GCN5 + LSD1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## HDAC8 22d

In [125]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_HDAC8_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x for x in dat.obs.index]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('[1-3]', '')
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('treat', '22d').str.replace('c', 'C')

In [126]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [127]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = '22d', cluster_B = 'Control', cohort = 'MV4-11', drug = '22d', target = 'HDAC8')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## SYK Entosplentinib

In [128]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_SYK_Results.csv")
dat = dat[:12]
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[2] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [129]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [130]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Entospletinib', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Entospletinib', target = 'SYK')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## Hypometh Azacitidine

In [131]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_hypometh_azacytidine_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]

In [132]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [133]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'Azacytidine', cluster_B = 'Control', cohort = 'OCI-AML3', drug = '5-Aza', target = 'Hypometh')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## Pyrimidine Synthesis AlCAr

In [134]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_Pyrimidine_Synthesis_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split(' ')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('[0-9]','').str.replace('ctrl', 'Control')

In [135]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [136]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'AICAr', cluster_B = 'Control', cohort = 'U-937', drug = 'AlCAr', target = 'Pyrimidine Synthesis')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## LSD1 INCB059872

In [137]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_LSD1_INCB059872_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [138]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [139]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'INCB059872', cluster_B = 'Control', cohort = 'OCI-AML3', drug = 'INCB059872', target = 'LSD1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## MYB AT HA

In [140]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_MYB_AT_HA_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control').str.replace('[0-9]', '')

In [141]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [142]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'AT', cluster_B = 'Control', cohort = 'THP-1', drug = 'AT', target = 'MYB')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [143]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'HA', cluster_B = 'Control', cohort = 'THP-1', drug = 'HA', target = 'MYB')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## PRMT5 EZH2 GSK

In [144]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_PRMT5_EZH2_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [145]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [146]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK3186000A', cluster_B = 'Control', cohort = 'MOLM-13', drug = 'GSK3186000A', target = 'PRMT5')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [147]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK343', cluster_B = 'Control', cohort = 'MOLM-13', drug = 'GSK343', target = 'EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [148]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'GSK3186000A+GSK343', cluster_B = 'Control', cohort = 'MOLM-13', drug = 'GSK3186000A+GSK343', target = 'PRMT5 + EZH2')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## HDAC ATRA TSA

In [149]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_HDAC_Results.csv")
dat = dat[:12]
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [150]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [151]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA', cluster_B = 'Control', cohort = 'AP-1060', drug = 'ATRA', target = 'RARA')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [152]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'TSA', cluster_B = 'Control', cohort = 'AP-1060', drug = 'TSA', target = 'HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [153]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA+TSA', cluster_B = 'Control', cohort = 'AP-1060', drug = 'ATRA + TSA', target = 'RARA + HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [154]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_HDAC_Results.csv")
dat = dat[12:24]
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [155]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [156]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA', cluster_B = 'Control', cohort = 'HL60', drug = 'ATRA', target = 'RARA')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [157]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'TSA', cluster_B = 'Control', cohort = 'HL60', drug = 'TSA', target = 'HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [158]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA+TSA', cluster_B = 'Control', cohort = 'HL60', drug = 'ATRA + TSA', target = 'RARA + HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [159]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_HDAC_Results.csv")
dat = dat[24:]
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

Trying to set attribute `.obs` of view, copying.


In [160]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [161]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA', cluster_B = 'Control', cohort = 'NB-4', drug = 'ATRA', target = 'RARA')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [162]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'TSA', cluster_B = 'Control', cohort = 'NB-4', drug = 'TSA', target = 'HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [163]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'ATRA+TSA', cluster_B = 'Control', cohort = 'NB-4', drug = 'ATRA + TSA', target = 'RARA + HDAC')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## EZH UNC1999

In [164]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_EZH_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [165]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

In [166]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'UNC1999', cluster_B = 'Control', cohort = 'OCI-AML3', drug = 'UNC1999', target = 'EZH2/EZH1')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

## TOPO II Etoposide

In [167]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_F36P = dat[:6]
dat_F36P.X = (dat_F36P.X.transpose() / dat_F36P.X.sum(axis=1)).transpose()
dat_F36P.obs['Treatment'] = [x[2] for x in dat_F36P.obs.index.str.split('_')]
dat_F36P.obs['Treatment'] = dat_F36P.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [168]:
newdat = dat_F36P.to_df()
newdat['PC2'] = (dat_F36P.to_df() * PC2_loadings).sum(axis=1)
dat_F36P = ad.AnnData(newdat, dat_F36P.obs)
del(newdat)

In [169]:
df = get_scores(df, dat_F36P, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'F-36P', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [170]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_HL60 = dat[6:12]
dat_HL60.X = (dat_HL60.X.transpose() / dat_HL60.X.sum(axis=1)).transpose()
dat_HL60.obs['Treatment'] = [x[2] for x in dat_HL60.obs.index.str.split('_')]
dat_HL60.obs['Treatment'] = dat_HL60.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [171]:
newdat = dat_HL60.to_df()
newdat['PC2'] = (dat_HL60.to_df() * PC2_loadings).sum(axis=1)
dat_HL60 = ad.AnnData(newdat, dat_HL60.obs)
del(newdat)

In [172]:
df = get_scores(df, dat_HL60, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'HL-60', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [173]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_Kasumi1 = dat[12:18]
dat_Kasumi1.X = (dat_Kasumi1.X.transpose() / dat_Kasumi1.X.sum(axis=1)).transpose()
dat_Kasumi1.obs['Treatment'] = [x[2] for x in dat_Kasumi1.obs.index.str.split('_')]
dat_Kasumi1.obs['Treatment'] = dat_Kasumi1.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [174]:
newdat = dat_Kasumi1.to_df()
newdat['PC2'] = (dat_Kasumi1.to_df() * PC2_loadings).sum(axis=1)
dat_Kasumi1 = ad.AnnData(newdat, dat_Kasumi1.obs)
del(newdat)

In [175]:
df = get_scores(df, dat_Kasumi1, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'Kasumi-1', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [176]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_MOLM13 = dat[18:24]
dat_MOLM13.X = (dat_MOLM13.X.transpose() / dat_MOLM13.X.sum(axis=1)).transpose()
dat_MOLM13.obs['Treatment'] = [x[2] for x in dat_MOLM13.obs.index.str.split('_')]
dat_MOLM13.obs['Treatment'] = dat_MOLM13.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [177]:
newdat = dat_MOLM13.to_df()
newdat['PC2'] = (dat_MOLM13.to_df() * PC2_loadings).sum(axis=1)
dat_MOLM13 = ad.AnnData(newdat, dat_MOLM13.obs)
del(newdat)

In [178]:
df = get_scores(df, dat_MOLM13, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'MOLM-13', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [179]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_MonoMac6 = dat[24:30]
dat_MonoMac6.X = (dat_MonoMac6.X.transpose() / dat_MonoMac6.X.sum(axis=1)).transpose()
dat_MonoMac6.obs['Treatment'] = [x[2] for x in dat_MonoMac6.obs.index.str.split('_')]
dat_MonoMac6.obs['Treatment'] = dat_MonoMac6.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [180]:
newdat = dat_MonoMac6.to_df()
newdat['PC2'] = (dat_MonoMac6.to_df() * PC2_loadings).sum(axis=1)
dat_MonoMac6 = ad.AnnData(newdat, dat_MonoMac6.obs)
del(newdat)

In [181]:
df = get_scores(df, dat_MonoMac6, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'MonoMac-6', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [182]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_MV411 = dat[30:36]
dat_MV411.X = (dat_MV411.X.transpose() / dat_MV411.X.sum(axis=1)).transpose()
dat_MV411.obs['Treatment'] = [x[2] for x in dat_MV411.obs.index.str.split('_')]
dat_MV411.obs['Treatment'] = dat_MV411.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [183]:
newdat = dat_MV411.to_df()
newdat['PC2'] = (dat_MV411.to_df() * PC2_loadings).sum(axis=1)
dat_MV411 = ad.AnnData(newdat, dat_MV411.obs)
del(newdat)

In [184]:
df = get_scores(df, dat_MV411, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'MV4-11', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [185]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_NB4 = dat[36:42]
dat_NB4.X = (dat_NB4.X.transpose() / dat_NB4.X.sum(axis=1)).transpose()
dat_NB4.obs['Treatment'] = [x[2] for x in dat_NB4.obs.index.str.split('_')]
dat_NB4.obs['Treatment'] = dat_NB4.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [186]:
newdat = dat_NB4.to_df()
newdat['PC2'] = (dat_NB4.to_df() * PC2_loadings).sum(axis=1)
dat_NB4 = ad.AnnData(newdat, dat_NB4.obs)
del(newdat)

In [187]:
df = get_scores(df, dat_NB4, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'NB-4', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [188]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_NOMO1 = dat[42:48]
dat_NOMO1.X = (dat_NOMO1.X.transpose() / dat_NOMO1.X.sum(axis=1)).transpose()
dat_NOMO1.obs['Treatment'] = [x[2] for x in dat_NOMO1.obs.index.str.split('_')]
dat_NOMO1.obs['Treatment'] = dat_NOMO1.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [189]:
newdat = dat_NOMO1.to_df()
newdat['PC2'] = (dat_NOMO1.to_df() * PC2_loadings).sum(axis=1)
dat_NOMO1 = ad.AnnData(newdat, dat_NOMO1.obs)
del(newdat)

In [190]:
df = get_scores(df, dat_NOMO1, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'NOMO-1', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [191]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_OCIAML2 = dat[48:54]
dat_OCIAML2.X = (dat_OCIAML2.X.transpose() / dat_OCIAML2.X.sum(axis=1)).transpose()
dat_OCIAML2.obs['Treatment'] = [x[2] for x in dat_OCIAML2.obs.index.str.split('_')]
dat_OCIAML2.obs['Treatment'] = dat_OCIAML2.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [192]:
newdat = dat_OCIAML2.to_df()
newdat['PC2'] = (dat_OCIAML2.to_df() * PC2_loadings).sum(axis=1)
dat_OCIAML2 = ad.AnnData(newdat, dat_OCIAML2.obs)
del(newdat)

In [193]:
df = get_scores(df, dat_OCIAML2, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'OCI-AML2', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [194]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_OCIAML3 = dat[54:60]
dat_OCIAML3.X = (dat_OCIAML3.X.transpose() / dat_OCIAML3.X.sum(axis=1)).transpose()
dat_OCIAML3.obs['Treatment'] = [x[2] for x in dat_OCIAML3.obs.index.str.split('_')]
dat_OCIAML3.obs['Treatment'] = dat_OCIAML3.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [195]:
newdat = dat_OCIAML3.to_df()
newdat['PC2'] = (dat_OCIAML3.to_df() * PC2_loadings).sum(axis=1)
dat_OCIAML3 = ad.AnnData(newdat, dat_OCIAML3.obs)
del(newdat)

In [196]:
df = get_scores(df, dat_OCIAML3, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'OCI-AML3', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

In [197]:
dat = load_fractions("Data/Datasets/Cell Lines/CIBERSORTx_TOPOII_Results.csv")
dat_THP1 = dat[60:]
dat_THP1.X = (dat_THP1.X.transpose() / dat_THP1.X.sum(axis=1)).transpose()
dat_THP1.obs['Treatment'] = [x[2] for x in dat_THP1.obs.index.str.split('_')]
dat_THP1.obs['Treatment'] = dat_THP1.obs['Treatment'].str.replace('VP16','Etoposide').str.replace('CTR', 'Control')

Trying to set attribute `.obs` of view, copying.


In [198]:
newdat = dat_THP1.to_df()
newdat['PC2'] = (dat_THP1.to_df() * PC2_loadings).sum(axis=1)
dat_THP1 = ad.AnnData(newdat, dat_THP1.obs)
del(newdat)

In [199]:
df = get_scores(df, dat_THP1, classification = 'Treatment', cluster_A = 'Etoposide', cluster_B = 'Control', cohort = 'THP-1', drug = 'Etoposide', target = 'TOPO II')

Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Quiescent-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Mono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ProMono-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cDC-like-Abs.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'GMP-like-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Cycle-Rel.Category' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'LSPC-Primed-Rel.Cate

***

# Summarize 

In [200]:
df['CellType'] = df['names'].astype('category').cat.set_categories(['LSPC-Quiescent', 'LSPC-Primed', 'LSPC-Cycle', 'GMP-like', 'ProMono-like', 'Mono-like', 'cDC-like', 'PC2'])

In [201]:
df['logPval_dir'] = np.where(df['scores'] < 0, np.log(df['pvals']), -np.log(df['pvals']))
df['Treatment'] = np.where(df['Cohort']=="Primary", df['Drug'], df['Drug'] + " [" + df['Cohort'].str.replace('Primary ', '') + "]")

In [202]:
pd.set_option('display.max_rows', 1500)
df.CellType = df.names

In [203]:
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/CIBERSORTx/Drug_Cell_Line_pvalues.csv')
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/Complex Heatmap/Data/Drug_Cell_Line_pvalues.csv')
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/Complex Heatmap/UMAP/Drug_Cell_Line_pvalues.csv')
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/Stouffer Method/Drug_Cell_Line_pvalues.csv')